In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.9.1
sys.version_info(major=3, minor=9, micro=7, releaselevel='final', serial=0)
matplotlib 3.5.2
numpy 1.23.0
pandas 1.4.3
sklearn 1.1.1
tensorflow 2.9.1
keras.api._v2.keras 2.9.0


In [2]:
# https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt
input_filepath = "./shakespeare.txt"
text = open(input_filepath, 'r').read()

print(len(text))
print(text[0:100])

1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [3]:
# 1. generate vocab
# 2. build mapping char->id
# 3. data -> id_data  把数据都转为id
# 4. abcd -> bcd<eos>  预测下一个字符生成的模型，也就是输入是a，输出就是b

# 去重，留下独立字符，并排序
vocab = sorted(set(text))
print(len(vocab))
print(vocab)

65
['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [4]:
# enumerate的作用
for idx,char in enumerate(['how','are','you']):
    print(idx,char)

0 how
1 are
2 you


In [5]:
# 每个字符都编好号，enumerate对每一个位置编号，生成的是列表中是元组
char2idx = {char:idx for idx, char in enumerate(vocab)}  # 字典生成式
print(char2idx)

{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}


In [11]:
# 把vocab从列表变为ndarray
idx2char = np.array(vocab)
print(idx2char)
print(idx2char[3])

['\n' ' ' '!' '$' '&' "'" ',' '-' '.' '3' ':' ';' '?' 'A' 'B' 'C' 'D' 'E'
 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R' 'S' 'T' 'U' 'V' 'W'
 'X' 'Y' 'Z' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j' 'k' 'l' 'm' 'n' 'o'
 'p' 'q' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y' 'z']
$


In [7]:
# 把字符都转换为id
text_as_int = np.array([char2idx[c] for c in text])
print(text_as_int.shape)
print(len(text_as_int))
print(text_as_int[0:10])
print(text[0:10])

(1115394,)
1115394
[18 47 56 57 58  1 15 47 58 47]
First Citi


# 把莎士比亚文集分成一个一个的样本

In [12]:
# 把输入和输出分配好
def split_input_target(id_text):
    """
    abcde -> abcd, bcde,输入是abcd，输出是bcde（也是目标值）
    """
    return id_text[0:-1], id_text[1:]

# 把id text转换为dataset
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
for ch_id in char_dataset.take(2):
    print(ch_id, idx2char[ch_id.numpy()])
print('-'*50)

seq_length = 100
# 做一个batch，这个batch是把字变为句子，一个句子是101个字符
seq_dataset = char_dataset.batch(seq_length + 1,  # seq_length + 1目的是我们输入是5个字符时，输出是4
                                 drop_remainder = True)  # drop_remainder是最后不够就丢掉
# seq_dataset 每一个都是句子，对应id，取两个句子看看
for seq_id in seq_dataset.take(2):
    print(seq_id)
    print(repr(''.join(idx2char[seq_id.numpy()])))

tf.Tensor(18, shape=(), dtype=int32) F
tf.Tensor(47, shape=(), dtype=int32) i
--------------------------------------------------
tf.Tensor(
[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43
 39 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49
  6  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10
  0 37 53 59  1], shape=(101,), dtype=int32)
'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
tf.Tensor(
[39 56 43  1 39 50 50  1 56 43 57 53 50 60 43 42  1 56 39 58 46 43 56  1
 58 53  1 42 47 43  1 58 46 39 52  1 58 53  1 44 39 51 47 57 46 12  0  0
 13 50 50 10  0 30 43 57 53 50 60 43 42  8  1 56 43 57 53 50 60 43 42  8
  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 18 47 56 57 58  6  1
 63 53 59  1 49], shape=(101,), dtype=int32)
'are all resolved rather to die than to famish?\n\nAll:\nResolve

In [13]:
# 然后通过split_input_target函数来对seq_dataset做映射，得到输入，输出
seq_dataset = seq_dataset.map(split_input_target)

for item_input, item_output in seq_dataset.take(1):
    print(item_input)
    print(item_output)
print('-'*50)
print(seq_dataset)

tf.Tensor(
[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43
 39 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49
  6  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10
  0 37 53 59], shape=(100,), dtype=int32)
tf.Tensor(
[47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43  1
 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43 39
 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49  6
  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0
 37 53 59  1], shape=(100,), dtype=int32)
--------------------------------------------------
<MapDataset element_spec=(TensorSpec(shape=(100,), dtype=tf.int32, name=None), TensorSpec(shape=(100,), dtype=tf.int32, name=None))>


In [14]:
1115394//101//64

172

In [15]:
batch_size = 64
buffer_size = 10000
# 这个batch是真正的batch，上一个batch是把字变为句子,buffer_size是从数据集拿多少元素
seq_dataset = seq_dataset.shuffle(buffer_size).batch(
    batch_size, drop_remainder=True)
print(seq_dataset)

<BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int32, name=None), TensorSpec(shape=(64, 100), dtype=tf.int32, name=None))>


In [16]:
vocab_size = len(vocab)
embedding_dim = 256  # 资料比较小，所以dim可以设大一些
rnn_units = 1024

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = keras.models.Sequential([
        keras.layers.Embedding(vocab_size, embedding_dim,
                               batch_input_shape = [batch_size, None]),
        keras.layers.SimpleRNN(units = rnn_units,
                               stateful = True,  # 是否把上一批最后返回的状态添加到下一批作为输入
                               recurrent_initializer = 'glorot_uniform',
                               return_sequences = True),  # return_sequences是指要返回一个序列，也就是所有输出，而不是最后一个
        # 全连接层，为什么最后一层全连接层的输出是vocab_size
        keras.layers.Dense(vocab_size),
    ])
    return model

model = build_model(
    vocab_size = vocab_size,
    embedding_dim = embedding_dim,
    rnn_units = rnn_units,
    batch_size = batch_size)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           16640     
                                                                 
 simple_rnn (SimpleRNN)      (64, None, 1024)          1311744   
                                                                 
 dense (Dense)               (64, None, 65)            66625     
                                                                 
Total params: 1,395,009
Trainable params: 1,395,009
Non-trainable params: 0
_________________________________________________________________


In [17]:
256*1024+1024*1024+1024

1311744

In [18]:
model.variables

[<tf.Variable 'embedding/embeddings:0' shape=(65, 256) dtype=float32, numpy=
 array([[ 0.03814925, -0.04826446, -0.0155986 , ...,  0.01227839,
         -0.0308719 ,  0.02873104],
        [-0.02812566, -0.02933854, -0.02048625, ...,  0.03362657,
         -0.02218577, -0.02646294],
        [-0.01404297,  0.04445117,  0.02927054, ..., -0.00314341,
          0.0460621 ,  0.03481469],
        ...,
        [ 0.02241622, -0.02110145,  0.0177743 , ..., -0.02307477,
          0.0028703 , -0.00022823],
        [-0.0442313 , -0.03920607,  0.02714305, ...,  0.04576707,
          0.00936097,  0.02695459],
        [ 0.04614358, -0.0255782 ,  0.03240695, ...,  0.01837621,
         -0.02727146, -0.02189356]], dtype=float32)>,
 <tf.Variable 'simple_rnn/simple_rnn_cell/kernel:0' shape=(256, 1024) dtype=float32, numpy=
 array([[ 0.05897592,  0.03295162,  0.01980956, ...,  0.00078882,
          0.00958551, -0.03069432],
        [ 0.05996761, -0.05616711,  0.03777836, ...,  0.05177031,
         -0.05729948

In [19]:
for input_example_batch, target_example_batch in seq_dataset.take(1):
    # 把model当函数来用，实际是调用类的call方法
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape)  # 这里是没有训练时，得到的一句预测输出

(64, 100, 65)


In [20]:
example_batch_predictions[0][0]

<tf.Tensor: shape=(65,), dtype=float32, numpy=
array([-7.5258398e-03, -6.2076703e-02, -2.0891201e-02,  8.0272416e-04,
        9.4193229e-03, -1.0665815e-02,  6.9902833e-03, -3.2273561e-02,
        6.1986134e-03,  1.9129405e-02,  1.6311502e-02,  3.2471389e-02,
        1.1829180e-02, -2.2169722e-02, -3.6852963e-02,  1.7297778e-02,
        8.8479863e-03,  4.6639517e-03, -7.5456843e-02,  6.4997934e-03,
        4.5748454e-02,  8.1170890e-03, -2.9248808e-02, -4.7551151e-04,
        4.5167501e-03, -1.8116293e-02, -1.8133435e-02, -5.2567753e-03,
       -2.0027686e-02,  7.7534132e-03, -1.7272785e-02,  5.0383992e-03,
        1.7964093e-02, -9.5565347e-03, -6.3367062e-03, -2.8726483e-02,
       -1.3765996e-02,  1.0582377e-02, -1.4698234e-02,  1.1079634e-02,
       -9.7004715e-03, -1.0353370e-02, -9.6735563e-03,  4.2993031e-02,
        8.0178026e-03, -1.9737933e-02, -7.0946049e-03, -1.4985650e-02,
        9.7375363e-05, -2.6357211e-02, -1.7786074e-02, -2.8135069e-02,
       -2.0472068e-02, -2.4858

In [21]:
example_batch_predictions[0]

<tf.Tensor: shape=(100, 65), dtype=float32, numpy=
array([[-0.00752584, -0.0620767 , -0.0208912 , ...,  0.02083052,
         0.02569194, -0.03518045],
       [ 0.05406422, -0.02065535,  0.05660959, ...,  0.03348031,
         0.03354893,  0.06156464],
       [ 0.08759753,  0.00416853,  0.02999344, ...,  0.00698912,
         0.07848426,  0.04821686],
       ...,
       [-0.2676754 , -0.04074209, -0.0514502 , ..., -0.07857621,
         0.30195117, -0.05222221],
       [ 0.19079989, -0.28130484, -0.01265898, ...,  0.00615522,
        -0.04481944, -0.06135776],
       [-0.2586094 , -0.06412364, -0.26018357, ..., -0.13282579,
        -0.00838639,  0.18280618]], dtype=float32)>

In [22]:
# random sampling.
# greedy, random.
# logits是计算分类任务之前，没有经过softmax的那个值就是logits，把第一个样本输进去
# print(example_batch_predictions[0][0])
sample_indices = tf.random.categorical(  # tf.random.categorical从分类分布中抽取样本,随机是为了每次写文章有差异
    logits = example_batch_predictions[0], num_samples = 1, seed = 1)
print(sample_indices)  # 这里的维度是（100,1）
print('-'*100)
# (100, 1) -> (100, )  调用squeeze 去除1的维度，变为100的向量
sample_indices = tf.squeeze(sample_indices, axis = -1)
print(sample_indices)

----------------------------------------------------------------------------------------------------
tf.Tensor(
[[59]
 [26]
 [44]
 [45]
 [ 9]
 [18]
 [13]
 [30]
 [39]
 [25]
 [37]
 [59]
 [26]
 [58]
 [52]
 [ 7]
 [32]
 [31]
 [43]
 [40]
 [42]
 [35]
 [42]
 [24]
 [25]
 [47]
 [32]
 [ 1]
 [57]
 [39]
 [12]
 [17]
 [ 3]
 [19]
 [61]
 [ 7]
 [ 1]
 [64]
 [57]
 [50]
 [38]
 [38]
 [14]
 [ 8]
 [ 2]
 [10]
 [31]
 [40]
 [60]
 [52]
 [39]
 [34]
 [38]
 [64]
 [16]
 [35]
 [15]
 [42]
 [ 5]
 [ 5]
 [38]
 [60]
 [41]
 [39]
 [39]
 [47]
 [54]
 [22]
 [51]
 [53]
 [10]
 [44]
 [22]
 [46]
 [19]
 [ 7]
 [ 8]
 [20]
 [47]
 [63]
 [43]
 [14]
 [16]
 [13]
 [11]
 [26]
 [30]
 [20]
 [61]
 [ 1]
 [43]
 [35]
 [33]
 [57]
 [21]
 [13]
 [64]
 [64]
 [ 9]
 [29]], shape=(100, 1), dtype=int64)
----------------------------------------------------------------------------------------------------
tf.Tensor(
[59 26 44 45  9 18 13 30 39 25 37 59 26 58 52  7 32 31 43 40 42 35 42 24
 25 47 32  1 57 39 12 17  3 19 61  7  1 64 57 50 38 38 14  8  2 10 31 40

In [24]:
# 理解random.categorical，虽然是随机的，但是还是偏向于概率较大的值
# num_samples，意味着我们搞几次抽样
for i in tf.range(5):
    samples = tf.random.categorical([[4.0,2.0,2.0,2.0,1.0]], 3)  # 3是num_samples
    tf.print(samples)

[[0 0 1]]
[[0 0 0]]
[[0 0 2]]
[[0 0 0]]
[[3 0 0]]


In [25]:
print("Input: ", repr("".join(idx2char[input_example_batch[0]])))
print('-'*50)
print("Output: ", repr("".join(idx2char[target_example_batch[0]])))
print('-'*50)
print("Predictions: ", repr("".join(idx2char[sample_indices])))

Input:  ' reign, but earth and dust?\nAnd, live we how we can, yet die we must.\n\nSOMERSET:\nAh, Warwick, Warwic'
--------------------------------------------------
Output:  'reign, but earth and dust?\nAnd, live we how we can, yet die we must.\n\nSOMERSET:\nAh, Warwick, Warwick'
--------------------------------------------------
Predictions:  "uNfg3FARaMYuNtn-TSebdWdLMiT sa?E$Gw- zslZZB.!:SbvnaVZzDWCd''ZvcaaipJmo:fJhG-.HiyeBDA;NRHw eWUsIAzz3Q"


In [26]:
# from_logits是否预期为对数张量。默认情况下，我们假设对概率分布进行编码
# logits表示网络的直接输出 。没经过sigmoid或者softmax的概率化
# from_logits=False就表示把已经概率化了的输出，重新映射回原值。log（p/(1-p)）
def loss(labels, logits):
    return keras.losses.sparse_categorical_crossentropy(
        labels, logits, from_logits=True)

model.compile(optimizer = 'adam', loss = loss)
example_loss = loss(target_example_batch, example_batch_predictions)
print(example_loss.shape)
print(example_loss.numpy().mean())  # 看下样例的loss

(64, 100)
4.1856117


In [ ]:
# 定义一个文件夹，保存模型
output_dir = "./text_generation_checkpoints"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
checkpoint_prefix = os.path.join(output_dir, 'ckpt_{epoch}')  # epoch是每一次epoch
checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_prefix,
    save_weights_only = True)  #     # 只保存权重的值

epochs = 100
history = model.fit(seq_dataset, epochs = epochs,
                    callbacks = [checkpoint_callback])


Epoch 1/100
172/172 [==============================] - 195s 1s/step - loss: 2.7963
Epoch 2/100
172/172 [==============================] - 221s 1s/step - loss: 2.0113
Epoch 3/100
172/172 [==============================] - 248s 1s/step - loss: 1.8067
Epoch 4/100
172/172 [==============================] - 241s 1s/step - loss: 1.6766
Epoch 5/100
172/172 [==============================] - 216s 1s/step - loss: 1.5900
Epoch 6/100
172/172 [==============================] - 189s 1s/step - loss: 1.5278
Epoch 7/100
172/172 [==============================] - 178s 1s/step - loss: 1.4836
Epoch 8/100
172/172 [==============================] - 163s 950ms/step - loss: 1.4484
Epoch 9/100
172/172 [==============================] - 160s 932ms/step - loss: 1.4212
Epoch 10/100
172/172 [==============================] - 169s 981ms/step - loss: 1.3962
Epoch 11/100
172/172 [==============================] - 169s 984ms/step - loss: 1.3755
Epoch 12/100
172/172 [==============================] - 164s 955ms/step -

In [26]:
output_dir = "./text_generation_checkpoints"
tf.train.latest_checkpoint(output_dir)

'./text_generation_checkpoints/ckpt_99'

In [27]:
output_dir = "./text_generation_checkpoints"
model2 = build_model(vocab_size,
                     embedding_dim,
                     rnn_units,
                     batch_size = 1)  # 参数数量不受batch_size影响
model2.load_weights(tf.train.latest_checkpoint(output_dir))
# 1是一个样本，None是可以变长序列
# model2.build(tf.TensorShape([1, None]))
# 下面是文本生成的流程
# start ch sequence A, 
# A -> model -> b  A放入模型后得到b
# A.append(b) -> B
# B(Ab) -> model -> c
# B.append(c) -> C
# C(Abc) -> model -> ...
model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (1, None, 1024)           1311744   
_________________________________________________________________
dense_1 (Dense)              (1, None, 65)             66625     
Total params: 1,395,009
Trainable params: 1,395,009
Non-trainable params: 0
_________________________________________________________________


In [27]:
i=[18,24,36,78]
tf.expand_dims(i,0)  # 扩维

<tf.Tensor: shape=(1, 4), dtype=int32, numpy=array([[18, 24, 36, 78]])>

In [5]:
a=np.array([[33],[23],[12],[34],[28]])
print(a[-1])
print(a[-1,0])
# idx2char[a[-1,0]]

[28]
28


In [38]:
# 定义一个函数来实现上面的文本生成流程
def generate_text(model, start_string, num_generate = 1000):
    # 这一次输出的是1维的,把字母变为对应的id
    input_eval = [char2idx[ch] for ch in start_string]
    print(input_eval)
    # 做一个维度扩展（在零轴位置）,为了满足输入的维度要求而做的
    input_eval = tf.expand_dims(input_eval, 0)
    print(input_eval)
    text_generated = []
    # 对model进行reset，连续调用的时候使用resets_states()
    model.reset_states()
    
    for _ in range(num_generate):
        # 1. model inference -> predictions
        # 2. sample -> ch -> text_generated.
        # 3. update input_eval
        
        # predictions : [batch_size, input_eval_len, vocab_size]
        predictions = model(input_eval)
        print(predictions.shape)  # 第一次是(1,5,65)
        # squeeze消掉 batch_size，变为predictions : [input_eval_len, vocab_size]
        predictions = tf.squeeze(predictions, 0)
        # predicted_ids: [input_eval_len, 1]
        # a b c -> b c d
#         print(predictions)
        # 把predictions : [input_eval_len, vocab_size]维度数据变为 1个维度,只拿最后一个
        predicted_id = tf.random.categorical(
            predictions, num_samples = 1)[-1, 0].numpy()  # 65个logits得到是什么类别
#         print(predicted_id)
        # 得到预测id后，放入text_generated
        text_generated.append(idx2char[predicted_id])
        # 下面这是是我们原来的公式,为什么没有append作为新的输入,因为那样比较低效
        # s, x -> rnn -> s', y
        input_eval = tf.expand_dims([predicted_id], 0)
#         print(input_eval)
    return start_string + ''.join(text_generated)

new_text = generate_text(model2, "All: ")
print(new_text)

[13, 50, 50, 10, 1]
tf.Tensor([[13 50 50 10  1]], shape=(1, 5), dtype=int32)
(1, 5, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)

(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)
(1, 1, 65)